In [ ]:
#基本的引入
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
#from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import random
#为了minibatch的功能
import torch.utils.data as Data
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset

In [ ]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(20)

In [ ]:
 # 1. 根据网络层的不同定义不同的初始化方式     
def weight_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        #nn.init.constant_(m.bias, 0) bias不要全初始化为0
        nn.init.normal_(m.bias, mean=0, std=1)
    # 也可以判断是否为conv2d，使用相应的初始化方式 
    elif isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
     # 是否为批归一化层
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)


In [ ]:
#改编版！新增monitor
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, monitor="val_acc", patience=7, verbose=False, delta=0):
        """
        Args:
            monitor (string): 可以选 "val_acc"or "val_loss"
                            
                            Default: "val_acc"
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.monitor=monitor
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.val_acc_max = 0
        self.delta = delta
    
    def __call__(self, val, model):
        if self.monitor=='val_loss':
            val_loss=val
            score = -val_loss

            if self.best_score is None:
                self.best_score = score
                self.save_checkpoint(val_loss, model)
            elif score < self.best_score + self.delta:
                self.counter += 1
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
                if self.counter >= self.patience:
                    self.early_stop = True
            else:
                self.best_score = score
                self.save_checkpoint(val_loss, model)
                self.counter = 0
        elif self.monitor=='val_acc':
            #这里的val是0-100之间的数。
            val_acc=val
            score = val_acc
            if self.best_score is None:
                self.best_score = score
                self.save_checkpoint(val_acc, model)
            elif score < self.best_score + self.delta:
                self.counter += 1
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
                if self.counter >= self.patience:
                    self.early_stop = True
            else:
                self.best_score = score
                self.save_checkpoint(val_acc, model)
                self.counter = 0   

    def save_checkpoint(self, val, model):
        '''Saves model when validation loss decrease.'''
        '''Saves model when validation accuracy increase.'''
        if self.monitor=='val_loss':
            if self.verbose:
                print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val:.6f}).  Saving model ...')           
            self.val_loss_min = val
        if self.monitor=='val_acc':
            if self.verbose:
                print(f'Validation accuracy increased ({self.val_acc_max:.6f}% --> {val:.6f}%).  Saving model ...')
            self.val_acc_max = val
        torch.save(model.state_dict(), 'checkpoint.pt')	# 这里会存储迄今最优模型的参数

In [ ]:
#新增train_acc,valid_acc
def train_model(model,device, patience, n_epochs):
    
    # to track the training loss as the model trains
    train_losses = []
    # to track the validation loss as the model trains
    valid_losses = []
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the training accuracy per epoch as the model trains
    train_accuracies = [] 
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = [] 
    # to track the validation accuracy per epoch as the model trains
    valid_accuracies = [] 
    # initialize the early_stopping object
    early_stopping = EarlyStopping("val_acc",patience=patience, verbose=True,delta=0.05)
    
    for epoch in range(1, n_epochs + 1):
 
        ###################
        # train the model #
        ###################
        model.train() # prep model for training
        train_correct=0
        for step, (X, y) in enumerate(train_loader):
            X, y = X.to(device), y.to(device)
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(X)
            # calculate the loss
            loss = loss_func(output, y)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # record training loss
            train_losses.append(loss.item())
            # update the train_correct label
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            train_correct += pred.eq(y.view_as(pred)).sum().item() 
        ######################    
        # validate the model #
        ######################
        model.eval() # prep model for evaluation
        # to mark the correct label as the model trains
        val_correct = 0
        for step, (X, y) in enumerate(valid_loader):
            X, y = X.to(device), y.to(device)
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(X)
            # calculate the loss
            loss = loss_func(output, y)
            # record validation loss
            valid_losses.append(loss.item())
            # update the val_correct label
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            val_correct += pred.eq(y.view_as(pred)).sum().item() 
        # print training/validation statistics 
        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        # calculate valid accuracy over an epoch
        train_acc=100. * train_correct / len(train_loader.dataset)
        valid_acc=100. * val_correct / len(valid_loader.dataset)
        train_accuracies.append(train_acc)
        valid_accuracies.append(valid_acc)
        
        epoch_len = len(str(n_epochs))
        
        print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'train_accuracy: {train_correct}/{len(train_loader.dataset)} ({train_acc:.5f})% ' +
                     f'\n    valid_loss: {valid_loss:.5f} ' +
                     f'valid_accuracy: {val_correct}/{len(valid_loader.dataset)} ({valid_acc:.5f})%')
        
        print(print_msg)
        
        # clear lists to track next epoch
        train_losses = []
        valid_losses = []
        # early_stopping needs the validation acc to check if it has incresed, 
        # and if it has, it will make a checkpoint of the current model
        early_stopping(valid_acc, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
    # load the last checkpoint with the best model
    model.load_state_dict(torch.load('checkpoint.pt'))
 
    return  model, avg_train_losses, avg_valid_losses,train_accuracies,valid_accuracies

In [ ]:
def test(model, device, test_loader,loss_func):
    model.eval()
    test_loss =[]
    correct = 0
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            output = model(X)
            test_loss.append( loss_func(output, y).item())
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()

    test_loss = np.average(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

导入数据

In [ ]:
transform1 = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform2 = transforms.Compose(
    [transforms.RandomResizedCrop((224, 224)),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
train_data = torchvision.datasets.ImageFolder("../input/caltech101/Caltech101/Caltech101/train",transform=transform1)
eval_data=torchvision.datasets.ImageFolder("../input/caltech101/Caltech101/Caltech101/eval",transform=transform1)
test_data=torchvision.datasets.ImageFolder("../input/caltech101/Caltech101/Caltech101/test",transform=transform1)

Num_workers=2
train_loader=Data.DataLoader(dataset=train_data,batch_size=32,
                             shuffle=True, num_workers=Num_workers)
valid_loader=Data.DataLoader(dataset=eval_data,batch_size=32,
                             shuffle=True, num_workers=Num_workers)
test_loader=Data.DataLoader(dataset=test_data,batch_size=64,
                             shuffle=True, num_workers=Num_workers)

源自网络，通过定义class做ResNet

In [ ]:
def Conv1(in_planes, places, stride=2):
    return nn.Sequential(
        nn.Conv2d(in_channels=in_planes,out_channels=places,kernel_size=7,stride=stride,padding=3, bias=False),
        nn.BatchNorm2d(places),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

class Bottleneck(nn.Module):
    def __init__(self,in_places,places, stride=1,downsampling=False, expansion = 4):
        super(Bottleneck,self).__init__()
        self.expansion = expansion
        self.downsampling = downsampling

        self.bottleneck = nn.Sequential(
            nn.Conv2d(in_channels=in_places,out_channels=places,kernel_size=1,stride=1, bias=False),
            nn.BatchNorm2d(places),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(places),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places*self.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(places*self.expansion),
        )

        if self.downsampling:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels=in_places, out_channels=places*self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(places*self.expansion)
            )
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        residual = x
        out = self.bottleneck(x)

        if self.downsampling:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self,blocks, num_classes=1000, expansion = 4):
        super(ResNet,self).__init__()
        self.expansion = expansion

        self.conv1 = Conv1(in_planes = 3, places= 64)

        self.layer1 = self.make_layer(in_places = 64, places= 64, block=blocks[0], stride=1)
        self.layer2 = self.make_layer(in_places = 256,places=128, block=blocks[1], stride=2)
        self.layer3 = self.make_layer(in_places=512,places=256, block=blocks[2], stride=2)
        self.layer4 = self.make_layer(in_places=1024,places=512, block=blocks[3], stride=2)

        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(2048,num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def make_layer(self, in_places, places, block, stride):
        layers = []
        layers.append(Bottleneck(in_places, places,stride, downsampling =True))
        for i in range(1, block):
            layers.append(Bottleneck(places*self.expansion, places))

        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.conv1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

def ResNet50():
    return ResNet([3, 4, 6, 3])

In [ ]:
lr = 0.001
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# create a new model with these weights
model = ResNet50().to(device)
model.apply(weight_init)
optimizer=torch.optim.Adam(model.parameters(),lr=lr,weight_decay=1e-5)
loss_func = nn.CrossEntropyLoss()  # the target label is NOT an one-hotted

In [ ]:
n_epochs=30
patience = 7
#optimizer=torch.optim.Adam(model.parameters(),lr=lr,weight_decay=1e-4)
model, train_loss, valid_loss,train_acc,valid_acc = train_model(model ,device, patience, n_epochs)

如果想要省事可以直接使用pytorch官方提供的ResNet,他还贴心的可以选择pretrain！效果非常棒！！！！！！！

In [ ]:
lr = 0.0001#lr=0.0001+batch_size=48,目前最好效果！
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# create a new model with these weights
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    model.to('cuda')
optimizer=torch.optim.Adam(model.parameters(),lr=lr,weight_decay=1e-5)
loss_func = nn.CrossEntropyLoss()  # the target label is NOT an one-hotted

In [ ]:
n_epochs=30
patience = 7
#optimizer=torch.optim.Adam(model.parameters(),lr=lr,weight_decay=1e-4)
model, train_loss, valid_loss,train_acc,valid_acc = train_model(model ,device, patience, n_epochs)

用了这个pretrain的ResNet，我的valid_acc 上了90%!!!幸福来的太突然。

In [ ]:
# visualize the loss as the network trained
fig = plt.figure(figsize=(10,8))
plt.plot(range(1,len(train_loss)+1),train_loss, label='Training Loss')
plt.plot(range(1,len(valid_loss)+1),valid_loss,label='Validation Loss')

# find position of lowest validation loss
minposs = valid_loss.index(min(valid_loss))+1 
#plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

plt.xlabel('epochs')
plt.ylabel('loss')
#plt.ylim(0, 0.5) # consistent scale
plt.xlim(0, len(train_loss)+1) # consistent scale
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig('loss_plot.png', bbox_inches='tight')

In [ ]:
# visualize the accuracy as the network trained
fig = plt.figure(figsize=(10,8))
plt.plot(range(1,len(train_acc)+1),train_acc, label='Training Accuracy')
plt.plot(range(1,len(valid_acc)+1),valid_acc,label='Validation Accuracy')

# find position of lowest validation loss
maxposs = valid_acc.index(max(valid_acc))+1 
plt.axvline(maxposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.ylim(0, 100) # consistent scale
plt.xlim(0, len(train_acc)+1) # consistent scale
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig('Accuracy_plot.png', bbox_inches='tight')

In [ ]:
test(model, device, test_loader,loss_func)